In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('C:\Eric\Data\m5-forecasting-accuracy'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:\Eric\Data\m5-forecasting-accuracy\calendar.csv
C:\Eric\Data\m5-forecasting-accuracy\M5_codes.txt
C:\Eric\Data\m5-forecasting-accuracy\sales_train_evaluation.csv
C:\Eric\Data\m5-forecasting-accuracy\sales_train_validation.csv
C:\Eric\Data\m5-forecasting-accuracy\sample_submission.csv
C:\Eric\Data\m5-forecasting-accuracy\sell_prices.csv


In [2]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

In [9]:
# Data manipulation
import pandas as pd
import numpy as np

# Modeling
import lightgbm as lgb

# Splitting data
from sklearn.model_selection import train_test_split

N_FOLDS = 5
MAX_EVALS = 5

#tr_last-max_lags = 1913 - 57 
#first_day = 1200 
1 if is_train else tr_last-max_lags, first_day 


start_day = max(1 if is_train  else tr_last-max_lags, first_day)

tr_last-max_lags = 1856
first_day = 1200

is_train : start_day = 1200
is_train no : start_day = 1856 

#i want to start this at 300 days before the end of traning dataset. so.. 1913 - 100 = 1813     

In [51]:
CAL_DTYPES={"event_name_1": "category", 
            "event_name_2": "category", 
            "event_type_1": "category", 
            "event_type_2": "category", 
            "weekday": "category", 
            'wm_yr_wk': 'int16', 
            "wday": "int16",
            "month": "int16", "year": "int16", 
            "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }

PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [52]:
h = 28 
max_lags = 57
tr_last = 1913

# tr_last - max_lags = 1856 
fday = datetime(2016,4, 25) 
fday

# dt period : 1713 ~ 1913 
# te period : 1856 ~ 1913  #1856 = 1913 - 57

def create_dt(is_train = True, nrows = None, first_day = 1713):
    
    # 데이터 읽고, 형 변환
    prices = pd.read_csv("C:\Eric\Data\m5-forecasting-accuracy\sell_prices.csv", dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()
            
    # 데이터 읽고, 형 변환     
    cal = pd.read_csv("C:\Eric\Data\m5-forecasting-accuracy\calendar.csv", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()
    
    # 
    start_day = max(1 if is_train  else tr_last-max_lags, first_day)
    
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]
    
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    
    dtype = {numcol:"float32" for numcol in numcols} 
    dtype.update({col: "category" for col in catcols if col != "id"})
    
    
    dt = pd.read_csv("C:\Eric\Data\m5-forecasting-accuracy\sales_train_validation.csv", 
                     nrows = nrows, usecols = catcols + numcols, dtype = dtype)
    
    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")
            dt[col] -= dt[col].min()
    
    if not is_train:
        for day in range(tr_last+1, tr_last+ 28 +1):
            dt[f"d_{day}"] = np.nan
    
    dt = pd.melt(dt,
                  id_vars = catcols,
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    
    dt = dt.merge(cal, on= "d", copy = False)
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    
    return dt

In [53]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    
    
    date_features = {
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [54]:
%%time

df = create_dt(is_train=True, first_day= 1713)
df.shape

Wall time: 9.26 s


In [57]:
%%time

create_fea(df)
df.shape

Wall time: 1min 3s


In [59]:
df.tail()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,...,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
6125106,FOODS_3_825_WI_3_validation,3046,6,9,2,2,d_1913,0.0,2016-04-24,11613,...,3.98,0.0,1.0,1.000000,0.714286,0.928571,1.250000,16,2,24
6125107,FOODS_3_826_WI_3_validation,3047,6,9,2,2,d_1912,1.0,2016-04-23,11613,...,1.28,0.0,2.0,0.857143,1.142857,1.035714,1.107143,16,2,23
6125108,FOODS_3_826_WI_3_validation,3047,6,9,2,2,d_1913,3.0,2016-04-24,11613,...,1.28,1.0,4.0,0.714286,1.571429,1.035714,1.250000,16,2,24
6125109,FOODS_3_827_WI_3_validation,3048,6,9,2,2,d_1912,0.0,2016-04-23,11613,...,1.00,0.0,0.0,0.000000,2.285714,1.821429,1.785714,16,2,23
6125110,FOODS_3_827_WI_3_validation,3048,6,9,2,2,d_1913,0.0,2016-04-24,11613,...,1.00,0.0,5.0,0.000000,2.428571,1.678571,1.964286,16,2,24


In [60]:
df.isnull().sum()

id                    0
item_id               0
dept_id               0
store_id              0
cat_id                0
state_id              0
d                     0
sales                 0
date                  0
wm_yr_wk              0
weekday               0
wday                  0
month                 0
year                  0
event_name_1          0
event_type_1          0
event_name_2          0
event_type_2          0
snap_CA               0
snap_TX               0
snap_WI               0
sell_price            0
lag_7            213430
lag_28           853720
rmean_7_7        396370
rmean_28_7      1036660
rmean_7_28      1036660
rmean_28_28     1676950
week                  0
quarter               0
mday                  0
dtype: int64

In [11]:
# Hyperparameter grid 1 "Master"
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
    
    
    
        "learning_rate" : 0.075,
    
    
    
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
#         "nthread" : 4
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 1200,
    'num_leaves': 128,
    "min_data_in_leaf": 100,
}


In [95]:
# Hyperparameter grid 2 
param_grid = {
    'boosting_type': ['gbdt' , 'dart'],
    'objective' : ['poisson'],
    'metric' :['rmse'],
    #시계열 objective function
    
    'num_leaves': list(range(20, 150)),
    
    
    'learning_rate': list(np.logspace(np.log10(0.005), np.log10(0.5), base = 10, num = 1000)),
    
    
    
    'subsample_for_bin': list(range(20000, 300000, 20000)),
    'min_child_samples': list(range(20, 500, 5)),
    'reg_alpha': list(np.linspace(0, 1)),
    'reg_lambda': list(np.linspace(0, 1)),
    'colsample_bytree': list(np.linspace(0.6, 1, 10)),
    'subsample': list(np.linspace(0.5, 1, 100)),
    'is_unbalance': [True, False]
}

In [96]:
random.seed(49)

# Randomly sample from dictionary
random_params = {k: random.sample(v, 1)[0] for k, v in param_grid.items()}
# Deal with subsample ratio
random_params['subsample'] = 1.0 if random_params['boosting_type'] == 'goss' else random_params['subsample']

random_params

{'boosting_type': 'gbdt',
 'objective': 'poisson',
 'metric': 'rmse',
 'num_leaves': 48,
 'learning_rate': 0.02299496045261219,
 'subsample_for_bin': 260000,
 'min_child_samples': 370,
 'reg_alpha': 0.6530612244897959,
 'reg_lambda': 0.061224489795918366,
 'colsample_bytree': 0.6,
 'subsample': 0.98989898989899,
 'is_unbalance': False}

In [98]:
def random_search(param_grid, max_evals = MAX_EVALS): #MAX_EVALS = 5 
    """Random search for hyperparameter optimization"""
    
    # Dataframe for results
    results = pd.DataFrame(columns = ['score', 'params', 'iteration'],
                                  index = list(range(MAX_EVALS)))
    
    # Keep searching until reach max evaluations
    for i in range(MAX_EVALS):
        
        # Choose random hyperparameters
        hyperparameters = {k: random.sample(v, 1)[0] for k, v in param_grid.items()}
        hyperparameters['subsample'] = 1.0 if hyperparameters['boosting_type'] == 'goss' else hyperparameters['subsample']

        # Evaluate randomly selected hyperparameters
        eval_results = objective(hyperparameters, i)
        
        results.loc[i, :] = eval_results
    
    # Sort with best score on top
    results.sort_values('score', ascending = False, inplace = True)
    results.reset_index(inplace = True)
    return results 

In [105]:
%%time
random_results = random_search(param_grid)

print('The best validation score was {:.5f}'.format(random_results.loc[0, 'score']))
print('\nThe best hyperparameters were:')

import pprint
pprint.pprint(random_results.loc[0, 'params'])

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


KeyboardInterrupt: 

In [104]:
def objective(hyperparameters, iteration):
    """Objective function for grid and random search. Returns
       the cross validation score from a set of hyperparameters."""
    
    # Number of estimators will be found using early stopping
    if 'n_estimators' in hyperparameters.keys():
        del hyperparameters['n_estimators']
    
     # Perform n_folds cross validation
    cv_results = lgb.cv(hyperparameters, train_data, num_boost_round = 10000, nfold = N_FOLDS, 
                        early_stopping_rounds = 100, metrics = 'rmse', seed = 42)
    
    # results to retun
    score = cv_results['auc-mean'][-1]
    estimators = len(cv_results['auc-mean'])
    hyperparameters['n_estimators'] = estimators 
    
    return [score, hyperparameters, iteration]

In [68]:
df.columns

Index(['id', 'item_id', 'dept_id', 'store_id', 'cat_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'lag_7', 'lag_28',
       'rmean_7_7', 'rmean_28_7', 'rmean_7_28', 'rmean_28_28', 'week',
       'quarter', 'mday'],
      dtype='object')

In [71]:
df.shape

(6125111, 31)

In [83]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = df.columns[~df.columns.isin(useless_cols)]
X_train = df[train_cols]
y_train = df["sales"]

In [88]:
%%time

np.random.seed(777)

fake_valid_inds = np.random.choice(X_train.index.values, 200_000, replace = False)

train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)

# Define Train & Validation Set 

train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds], 
                         categorical_feature=cat_feats, free_raw_data=False)

fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
                              categorical_feature=cat_feats,
                 free_raw_data=False)# This is a random sample, we're not gonna apply any time series train-test-split tricks here!

Wall time: 1.87 s


In [89]:
del df, X_train, y_train, fake_valid_inds,train_inds ; gc.collect()

77

In [ ]:
%%time



model = lgb.LGBMClassifier(**random_search_params, random_state = 42)

m_lgb = model.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=20) 

In [ ]:
m_lgb.save_model("model.lgb")

In [ ]:
te.loc[te.date == day, "sales"] = alpha*m_lgb.predict(tst)

In [77]:
X_train.shape

(6125111, 25)

In [78]:
y_train.shape

(6125111,)

In [81]:
X_train.tail()

,item_id,dept_id,store_id,cat_id,state_id,wday,month,year,event_name_1,event_type_1,...,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
6125106,3046,6,9,2,2,2,4,2016,0,0,...,3.98,0.0,1.0,1.000000,0.714286,0.928571,1.250000,16,2,24
6125107,3047,6,9,2,2,1,4,2016,0,0,...,1.28,0.0,2.0,0.857143,1.142857,1.035714,1.107143,16,2,23
6125108,3047,6,9,2,2,2,4,2016,0,0,...,1.28,1.0,4.0,0.714286,1.571429,1.035714,1.250000,16,2,24
6125109,3048,6,9,2,2,1,4,2016,0,0,...,1.00,0.0,0.0,0.000000,2.285714,1.821429,1.785714,16,2,23
6125110,3048,6,9,2,2,2,4,2016,0,0,...,1.00,0.0,5.0,0.000000,2.428571,1.678571,1.964286,16,2,24


In [80]:
y_train.tail()

6125106    0.0
6125107    1.0
6125108    3.0
6125109    0.0
6125110    0.0
Name: sales, dtype: float32

In [86]:
train

array([      0,       1,       2, ..., 5104257, 5104258, 5104259])

In [73]:
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(max_train_size=None, n_splits=5)

for train, test in tscv.split(df):
    print("TRAIN:", train, "TEST:", test)


TRAIN: [      0       1       2 ... 1020853 1020854 1020855] TEST: [1020856 1020857 1020858 ... 2041704 2041705 2041706]
TRAIN: [      0       1       2 ... 2041704 2041705 2041706] TEST: [2041707 2041708 2041709 ... 3062555 3062556 3062557]
TRAIN: [      0       1       2 ... 3062555 3062556 3062557] TEST: [3062558 3062559 3062560 ... 4083406 4083407 4083408]
TRAIN: [      0       1       2 ... 4083406 4083407 4083408] TEST: [4083409 4083410 4083411 ... 5104257 5104258 5104259]
TRAIN: [      0       1       2 ... 5104257 5104258 5104259] TEST: [5104260 5104261 5104262 ... 6125108 6125109 6125110]


In [ ]:
train_data = lgb.Dataset(train, label = )

In [ ]:
train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds], 
                         categorical_feature=cat_feats, free_raw_data=False)

In [10]:
# Dataframes for random and grid search
random_results = pd.DataFrame(columns = ['score', 'params', 'iteration'],
                              index = list(range(MAX_EVALS)))

grid_results = pd.DataFrame(columns = ['score', 'params', 'iteration'],
                              index = list(range(MAX_EVALS)))

In [62]:
import random
random.seed(49)

# Randomly sample from dictionary
random_params = {k: random.sample(v, 1)[0] for k, v in param_grid.items()}
# Deal with subsample ratio
random_params['subsample'] = 1.0 if random_params['boosting_type'] == 'goss' else random_params['subsample']

random_params

{'boosting_type': 'gbdt',
 'num_leaves': 108,
 'learning_rate': 0.03514122132154174,
 'subsample_for_bin': 40000,
 'min_child_samples': 225,
 'reg_alpha': 0.9795918367346939,
 'reg_lambda': 0.7142857142857142,
 'colsample_bytree': 0.9555555555555555,
 'subsample': 0.5303030303030303,
 'is_unbalance': True}

In [63]:
import csv

# Create file and open connection
out_file = 'random_search_trials.csv'
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)

# Write column names
headers = ['score', 'hyperparameters', 'iteration']
writer.writerow(headers)
of_connection.close()

In [64]:
def random_search(param_grid, out_file, max_evals = MAX_EVALS):
    """Random search for hyperparameter optimization. 
       Writes result of search to csv file every search iteration."""
    
    
    # Dataframe for results
    results = pd.DataFrame(columns = ['score', 'params', 'iteration'],
                                  index = list(range(MAX_EVALS)))
    for i in range(MAX_EVALS):
        
        # Choose random hyperparameters
        random_params = {k: random.sample(v, 1)[0] for k, v in param_grid.items()}
        random_params['subsample'] = 1.0 if random_params['boosting_type'] == 'goss' else random_params['subsample']

        # Evaluate randomly selected hyperparameters
        eval_results = objective(random_params, i)
        results.loc[i, :] = eval_results

        # open connection (append option) and write results
        of_connection = open(out_file, 'a')
        writer = csv.writer(of_connection)
        writer.writerow(eval_results)
        
        # make sure to close connection
        of_connection.close()
        
    # Sort with best score on top
    results.sort_values('score', ascending = False, inplace = True)
    results.reset_index(inplace = True)

    return results 

In [65]:
random_results = random_search(param_grid)

print('The best validation score was {:.5f}'.format(random_results.loc[0, 'score']))
print('\nThe best hyperparameters were:')

import pprint
pprint.pprint(random_results.loc[0, 'params'])

TypeError: random_search() missing 1 required positional argument: 'out_file'

In [ ]:
# Get the best parameters
random_search_params = random_results.loc[0, 'params']

# Create, train, test model
model = lgb.LGBMClassifier(**random_search_params, random_state = 42)
model.fit(train_features, train_labels)

preds = model.predict_proba(test_features)[:, 1]

print('The best model from random search scores {:.5f} ROC AUC on the test set.'.format(roc_auc_score(test_labels, preds)))

In [ ]:
random_results['params']